In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
import time


In [2]:


def normalized_cross_correlation_coefficient(image, template):
    """
    计算规范化交叉相关系数
    :param image: 图像
    :param template: 模板图像
    :return: 规范化交叉相关系数
    """
    # 计算图像和模板的平均值
    image_mean = np.mean(image)
    template_mean = np.mean(template)

    # 计算图像和模板的标准差
    image_std = np.std(image)
    template_std = np.std(template)

    # 计算图像和模板的乘积之和
    sum_product = np.sum((image - image_mean) * (template - template_mean))

    # 计算规范化交叉相关系数
    nccc = sum_product / ((image.size - 1) * image_std * template_std)

    return nccc

def template_matching(image, template):
    """
    进行模板匹配
    :param image: 图像
    :param template: 模板图像
    :return: 结果图像
    """
    begin = time.time()
    
    # 获取图像的高度和宽度
    height, width = image.shape[:2]
    template_height, template_width = template.shape[:2]

    # 创建结果图像，用于显示匹配结果
    result = np.empty((height - template_height + 1, width - template_width + 1))

    # 遍历每个像素，计算规范化交叉相关系数
    for y in range(height - template_height + 1):
        for x in range(width - template_width + 1):
            roi = image[y:y + template_height, x:x + template_width]
            result[y, x] = normalized_cross_correlation_coefficient(roi, template)
    end = time.time()
    print(end-begin)
    return result

In [3]:
def draw(img,h,w,th,tw,color):
    image_drawed = img.copy()
    image_drawed[h:h+5,w:w+tw] = color
    image_drawed[h+th-5:h+th,w:w+tw] = color
    image_drawed[h:h+th,w:w+5] = color
    image_drawed[h:h+th,w+tw-5:w+tw] = color
    image_drawed[(h+h+th) // 2,w:w+tw] = color
    image_drawed[h:h+th,(w + w+tw) // 2] = color
    return image_drawed

In [4]:
def center(h,w,th,tw):
    h = ((h + h) + th) // 2
    w = ((w + w) + tw) // 2
    return h,w

def center_diff(h,w,h1,w1):
    return ((h-h1) ** 2 + (w-w1) ** 2) ** 0.5 

In [5]:


# image = cv2.imread('Matching/circle/Panel1_circle1.bmp')
path = 'Matching/circle/'
out_path = 'Output/'
template = cv2.imread('Matching/pattern/Template_BorderCircle.bmp')
template_small = cv2.imread('Matching/pattern/Template_circle.bmp')
pry = 3

for file_name in  tqdm(os.listdir(path)):
    image = cv2.imread(path + file_name)
    image1 = cv2.resize(image, (image.shape[1]// pry, image.shape[0]// pry))
    template1 = cv2.resize(template, (template.shape[1]// pry, template.shape[0]// pry))
    template2 = cv2.resize(template_small, (template_small.shape[1]// pry, template_small.shape[0]// pry))

    nccc = template_matching(image1, template1)

    h, w = np.where(nccc == np.max(nccc))
    h, w = h[0], w[0]

    image_match = image1[h:h+template1.shape[1], w:w+template1.shape[0]].copy()

    image_drawed1 = draw(image,h*pry,w*pry,template.shape[1],template.shape[0],[255,0,255])

    center1_h,center1_w = center(h*pry,w*pry,template.shape[1],template.shape[0])


    nccc2 = template_matching(image_match,template2)

    h_small, w_small = np.where(nccc2 == np.max(nccc2))
    h_small, w_small = h_small[0], w_small[0]

    center2_h,center2_w = center(h*pry + h_small*pry,w*pry + w_small*pry,template_small.shape[1],template_small.shape[0])

    distance = center_diff(center1_h,center1_w,center2_h,center2_w)

    image_drawed2 = draw(image_drawed1,h*pry + h_small*pry,w*pry + w_small*pry,
                         template_small.shape[1],template_small.shape[0],[0,0,255])


    out = cv2.putText(image_drawed2, "distance: " + str(round(distance,4)), (40, 70), cv2.FONT_HERSHEY_SIMPLEX,
      2, (255, 255, 255), 5, cv2.LINE_AA)

    # print(image_drawed.shape)
    # plt.imshow(image_match[h_small:h_small+template2.shape[1], w_small:w_small+template2.shape[0]])
    plt.imsave((out_path + file_name).replace("bmp","png"),out)
#     plt.show()

  0%|                                                    | 0/16 [00:00<?, ?it/s]

9.390403985977173


  6%|██▊                                         | 1/16 [00:09<02:26,  9.79s/it]

0.2571568489074707
9.353968143463135


 12%|█████▌                                      | 2/16 [00:19<02:16,  9.77s/it]

...

0.25428009033203125
9.171227216720581


 94%|████████████████████████████████████████▎  | 15/16 [02:23<00:09,  9.54s/it]

0.25402188301086426
9.120733499526978


100%|███████████████████████████████████████████| 16/16 [02:33<00:00,  9.58s/it]

0.251727819442749


In [6]:
path = 'Matching/cross/'
out_path = 'Output/'
template = cv2.imread('Matching/pattern/Template_BorderCross.bmp')
template_small = cv2.imread('Matching/pattern/Template_cross.bmp')


for file_name in  tqdm(os.listdir(path)):
    image = cv2.imread(path + file_name)
    image1 = cv2.resize(image, (image.shape[1]// pry, image.shape[0]// pry))
    template1 = cv2.resize(template, (template.shape[1]// pry, template.shape[0]// pry))
    template2 = cv2.resize(template_small, (template_small.shape[1]// pry, template_small.shape[0]// pry))

    nccc = template_matching(image1, template1)

    h, w = np.where(nccc == np.max(nccc))
    h, w = h[0], w[0]

    image_match = image1[h:h+template1.shape[1], w:w+template1.shape[0]].copy()

    image_drawed1 = draw(image,h*pry,w*pry,template.shape[1],template.shape[0],[255,0,255])

    center1_h,center1_w = center(h*pry,w*pry,template.shape[1],template.shape[0])


    nccc2 = template_matching(image_match,template2)

    h_small, w_small = np.where(nccc2 == np.max(nccc2))
    h_small, w_small = h_small[0], w_small[0]

    center2_h,center2_w = center(h*pry + h_small*pry,w*pry + w_small*pry,template_small.shape[1],template_small.shape[0])

    distance = center_diff(center1_h,center1_w,center2_h,center2_w)

    image_drawed2 = draw(image_drawed1,h*pry + h_small*pry,w*pry + w_small*pry,
                         template_small.shape[1],template_small.shape[0],[0,0,255])


    out = cv2.putText(image_drawed2, "distance: " + str(round(distance,4)), (40, 70), cv2.FONT_HERSHEY_SIMPLEX,
      2, (255, 255, 255), 5, cv2.LINE_AA)

    # print(image_drawed.shape)
    # plt.imshow(image_match[h_small:h_small+template2.shape[1], w_small:w_small+template2.shape[0]])
    plt.imsave((out_path + file_name).replace("bmp","png"),out)

  0%|                                                    | 0/16 [00:00<?, ?it/s]

9.12691855430603


  6%|██▊                                         | 1/16 [00:09<02:22,  9.52s/it]

0.2529175281524658
9.129257678985596


 12%|█████▌                                      | 2/16 [00:19<02:13,  9.52s/it]

...

0.26418638229370117
9.097150325775146


 94%|████████████████████████████████████████▎  | 15/16 [02:23<00:09,  9.60s/it]

0.2679741382598877
9.116955995559692


100%|███████████████████████████████████████████| 16/16 [02:32<00:00,  9.54s/it]

0.26327967643737793
